# Read data

In [39]:
import pandas as pd
import os

In [40]:
path = "data/RAW_goldenqa_iteration1_oe_india_REVISED.tsv"
basename = os.path.basename(path)
basename = os.path.splitext(basename)[0]

qa_df = pd.read_csv(path, sep="\t", header=0)
qa_df.describe(include="all")

,question_id,question,correct_answer,comment
count,15.000000,15,15,1
unique,NaN,15,15,1
top,NaN,I have been applying different pesticides to c...,"Dear farmer friend, please follow the followin...",Note: Select varieties based on your region’s ...
freq,NaN,1,1,1
mean,8.000000,NaN,NaN,NaN
std,4.472136,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN
25%,4.500000,NaN,NaN,NaN
50%,8.000000,NaN,NaN,NaN
75%,11.500000,NaN,NaN,NaN


In [41]:
qa_df

,question_id,question,correct_answer,comment
0,1,I have been applying different pesticides to c...,"Dear farmer friend, please follow the followin...",NaN
1,2,I have been losing my crops and livestock due ...,Farmers can get timely compensation by enrolli...,NaN
2,3,I am interested in learning about crops and li...,To acquire adequate knowledge on crops and liv...,NaN
3,4,Which bean variety is suitable for dryland agr...,Following are the major bean varieties based o...,Note: Select varieties based on your region’s ...
4,5,If I am unable to acquire certified maize seed...,"Dear Farmer, avoid saving hybrid varieties as ...",NaN
5,6,Are chemical pesticides important and safe in ...,"Yes, chemical pesticides can be effective when...",NaN
6,7,How can I know that my soil is fertile?,"To assess soil fertility, request/acquire soil...",NaN
7,8,How can I best control weeds in my farm?,For best control of weeds adopt the following ...,NaN
8,9,How do organic and synthetic/inorganic fertili...,Inorganic fertilizers offer quick nutrient sup...,NaN
9,10,"Lately, the onset of rains has shown significa...",For better reesults and information the farmer...,NaN


# Set up API keys

In [42]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [43]:
def open_ended_prompt_simple(question, country):
    return (f"You provide smallholder farmers in {country} with agricultural advice and information. "
            f"Do not use more than 10 sentences.\n\nQuestion: {question}")

def open_ended_prompt_contextual(question, country):
    return (f"You provide smallholder farmers in {country} with high quality agricultural advice and information. "
            f"Please respond to the following question like an agronomist or agricultural extension agent would respond "
            f"to a smallholder farmer. Be factually correct, precise and make the response understandable for a smallholder farmer. "
            f"Do not use more than 10 sentences.\n\nQuestion: {question}")

def mcq_prompt(question, answer_options, country):
    return (f"You provide smallholder farmers in {country} with agricultural advice and information. "
            f"For the following multiple choice question, please select the correct answer(s) from options A to E and "
            f"provide a detailed rationale for your choice. There can be multiple correct answers.\n\n"
            f"Question: {question}\n\n"
            f"Answer options:\n{answer_options}\n\n"
            f"Your response must follow this format exactly:\n\n"
            f"Correct answer: [OPTION(S)]\n\n"
            f"Rationale: [Not longer than 10 sentences]")

In [44]:
import litellm

In [45]:
# set callbacks
litellm.success_callback = ["langfuse"]
litellm.failure_callback = ["langfuse"]

In [46]:
country = "India"

In [47]:
# model = "openai/gpt-4o"

# ids = []
# prompts = []
# answers = []

# for index, row in df.iterrows():
#     question = row["question"]
#     id = row["question_id"]
#     prompt = open_ended_prompt_contextual(question, country)

#     response = litellm.completion(
#         model=model,
#         messages=[
#             {"role": "user", "content": prompt}
#         ],
#     )

#     # Extract the answer from the response
#     answer = response["choices"][0]["message"]["content"]

#     # Append the id, prompt, and answer to the lists
#     ids.append(id)
#     prompts.append(prompt)
#     answers.append(answer)

# # Create a DataFrame from the lists
# results_df = pd.DataFrame({
#     "id": ids,
#     "prompt": prompts,
#     "answer": answers
# })

# # Save the DataFrame to a CSV file
# model_name = model.replace("/", "-")
# csv_path = os.path.join("results", f"{basename}-{model_name}.csv")
# print(f"Saving results to {csv_path}")
# results_df.to_csv(csv_path, index=False)

In [ ]:
def get_llm_answers(df, model, country, **kwargs):
    ids = []
    prompts = []
    answers = []

    for index, row in df.iterrows():
        question = row["question"]
        id = row["question_id"]
        prompt = open_ended_prompt_contextual(question, country)

        response = litellm.completion(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            **kwargs
        )

        # Extract the answer from the response
        answer = response["choices"][0]["message"]["content"]

        # Append the id, prompt, and answer to the lists
        ids.append(id)
        prompts.append(prompt)
        answers.append(answer)

    # Create a DataFrame from the lists
    results_df = pd.DataFrame({
        "id": ids,
        "prompt": prompts,
        "answer": answers
    })

    # Save the DataFrame to a CSV file
    model_name = model.replace("/", "-").replace(".", "-")
    csv_path = os.path.join("results", f"{basename}-{model_name}.csv")
    print(f"Saving results to {csv_path}")
    results_df.to_csv(csv_path, index=False)
    return results_df

# OpenAI

In [ ]:
gpt_4o_df = get_llm_answers(qa_df, "openai/gpt-4o-2024-08-06", country)

Saving results to results/RAW_goldenqa_iteration1_oe_india_REVISED-openai-gpt-4o-2024-08-06.csv


In [15]:
gpt_4o_df

,id,prompt,answer
0,1,You provide smallholder farmers in India with ...,I'm sorry to hear you're facing this issue. He...
1,2,You provide smallholder farmers in India with ...,I'm sorry to hear about the impact of droughts...
2,3,You provide smallholder farmers in India with ...,To learn more about crops and livestock produc...
3,4,You provide smallholder farmers in India with ...,"For a dryland agro-ecological zone, it's impor..."
4,5,You provide smallholder farmers in India with ...,"If certified maize seeds are not available, it..."
5,6,You provide smallholder farmers in India with ...,Chemical pesticides can play a key role in pro...
6,7,You provide smallholder farmers in India with ...,"To determine if your soil is fertile, you shou..."
7,8,You provide smallholder farmers in India with ...,Controlling weeds effectively involves a combi...
8,9,You provide smallholder farmers in India with ...,Both organic and synthetic (inorganic) fertili...
9,10,You provide smallholder farmers in India with ...,The variability in rainfall can indeed make pl...


In [ ]:
o1_df = get_llm_answers(qa_df, "openai/o1-2024-12-17", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/RAW_goldenqa_iteration1_oe_india_REVISED-openai-o1-2024-12-17.csv


In [14]:
o1_df

,id,prompt,answer
0,1,You provide smallholder farmers in India with ...,"First, identify the specific pest and confirm ..."
1,2,You provide smallholder farmers in India with ...,You should consider enrolling in government-ba...
2,3,You provide smallholder farmers in India with ...,Your local Krishi Vigyan Kendra (KVK) is a gre...
3,4,You provide smallholder farmers in India with ...,Cowpea (also called lobia) is one of the best ...
4,5,You provide smallholder farmers in India with ...,You can save and replant open-pollinated maize...
5,6,You provide smallholder farmers in India with ...,Chemical pesticides can help protect your crop...
6,7,You provide smallholder farmers in India with ...,A laboratory soil test is the most reliable wa...
7,8,You provide smallholder farmers in India with ...,"1. Use certified, weed-free seeds and plant th..."
8,9,You provide smallholder farmers in India with ...,"Organic fertilizers come from compost, manure,..."
9,10,You provide smallholder farmers in India with ...,Monitor local weather forecasts and rainfall d...


In [ ]:
o3_df = get_llm_answers(qa_df, "openai/o3-2025-04-16", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/RAW_goldenqa_iteration1_oe_india_REVISED-openai-o3-2025-04-16.csv


In [17]:
o3_df

,id,prompt,answer
0,1,You provide smallholder farmers in India with ...,1. First make sure you have correctly identifi...
1,2,You provide smallholder farmers in India with ...,1. The main way to receive compensation for cr...
2,3,You provide smallholder farmers in India with ...,1. Start with the Krishi Vigyan Kendra (KVK) i...
3,4,You provide smallholder farmers in India with ...,"For dryland conditions, use the French bean (v..."
4,5,You provide smallholder farmers in India with ...,Choose an open-pollinated composite (OPV) maiz...
5,6,You provide smallholder farmers in India with ...,"Yes, chemical pesticides can be useful tools f..."
6,7,You provide smallholder farmers in India with ...,1. The surest way to judge fertility is to sen...
7,8,You provide smallholder farmers in India with ...,"1. Start with clean seed and a well-prepared, ..."
8,9,You provide smallholder farmers in India with ...,"1. Organic sources like farmyard manure, compo..."
9,10,You provide smallholder farmers in India with ...,"1. Instead of fixing a calendar date, wait to ..."


In [ ]:
o4_mini_df = get_llm_answers(qa_df, "openai/o4-mini-2025-04-16", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/RAW_goldenqa_iteration1_oe_india_REVISED-openai-o4-mini-2025-04-16.csv


In [20]:
o4_mini_df

,id,prompt,answer
0,1,You provide smallholder farmers in India with ...,"1. First, correctly identify the pest species ..."
1,2,You provide smallholder farmers in India with ...,1. Enroll your crops in the Pradhan Mantri Fas...
2,3,You provide smallholder farmers in India with ...,1. To learn both crop and livestock production...
3,4,You provide smallholder farmers in India with ...,"You need a short‐duration, drought‐tolerant pu..."
4,5,You provide smallholder farmers in India with ...,Only open-pollinated (OPV) or composite maize ...
5,6,You provide smallholder farmers in India with ...,Chemical pesticides can be important for contr...
6,7,You provide smallholder farmers in India with ...,"1. Healthy soil is usually dark, crumbly and h..."
7,8,You provide smallholder farmers in India with ...,1. Begin with clean seed and avoid bringing we...
8,9,You provide smallholder farmers in India with ...,"1. Organic fertilizers (compost, farmyard manu..."
9,10,You provide smallholder farmers in India with ...,"1. Rather than fixed calendar dates, sow when ..."


# Anthropic

In [ ]:
claude_3_5_df = get_llm_answers(qa_df, "anthropic/claude-3-5-sonnet-20241022", country)

Saving results to results/RAW_goldenqa_iteration1_oe_india_REVISED-anthropic-claude-3-5-sonnet-20241022.csv


In [22]:
claude_3_5_df

,id,prompt,answer
0,1,You provide smallholder farmers in India with ...,Thank you for your question about pesticide ef...
1,2,You provide smallholder farmers in India with ...,I understand your concerns about crop and live...
2,3,You provide smallholder farmers in India with ...,I recommend visiting your nearest Krishi Vigya...
3,4,You provide smallholder farmers in India with ...,"For dryland farming, I recommend growing Black..."
4,5,You provide smallholder farmers in India with ...,"As an agricultural advisor, I recommend using ..."
5,6,You provide smallholder farmers in India with ...,Chemical pesticides can help protect your crop...
6,7,You provide smallholder farmers in India with ...,Namaste farmer! There are several simple ways ...
7,8,You provide smallholder farmers in India with ...,There are several effective ways to control we...
8,9,You provide smallholder farmers in India with ...,Let me explain the key differences between org...
9,10,You provide smallholder farmers in India with ...,"As an agricultural extension agent, I understa..."


In [ ]:
claude_3_7_df = get_llm_answers(qa_df, "anthropic/claude-3-7-sonnet-20250219", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/RAW_goldenqa_iteration1_oe_india_REVISED-anthropic-claude-3-7-sonnet-20250219.csv


In [12]:
claude_3_7_df

,id,prompt,answer
0,1,You provide smallholder farmers in India with ...,# Improving Your Pest Control Approach\n\nWhen...
1,2,You provide smallholder farmers in India with ...,# Compensation Options for Crop and Livestock ...
2,3,You provide smallholder farmers in India with ...,# Learning Resources for Crops and Livestock P...
3,4,You provide smallholder farmers in India with ...,# Bean Varieties for Dryland Farming\n\nFor dr...
4,5,You provide smallholder farmers in India with ...,# Maize Seed Saving Advice\n\nYou should save ...
5,6,You provide smallholder farmers in India with ...,# Chemical Pesticides in Farming: Important bu...
6,7,You provide smallholder farmers in India with ...,# How to Check Your Soil Fertility\n\nGood soi...
7,8,You provide smallholder farmers in India with ...,# Weed Control for Your Farm\n\nControlling we...
8,9,You provide smallholder farmers in India with ...,# Comparing Organic and Synthetic Fertilizers\...
9,10,You provide smallholder farmers in India with ...,# Adapting Planting Times to Variable Rainfall...


Deepseek models don't support the `reasoning_effort` parameter.

In [ ]:
deepseek_v3_df = get_llm_answers(qa_df, "deepseek/deepseek-chat", country, max_completion_tokens=25000)

Saving results to results/RAW_goldenqa_iteration1_oe_india_REVISED-deepseek-deepseek-chat.csv


In [23]:
deepseek_v3_df

,id,prompt,answer
0,1,You provide smallholder farmers in India with ...,"If pesticides are not working, first check if ..."
1,2,You provide smallholder farmers in India with ...,To get compensation for crop and livestock los...
2,3,You provide smallholder farmers in India with ...,You can visit your nearest **Krishi Vigyan Ken...
3,4,You provide smallholder farmers in India with ...,"For dryland farming in India, drought-tolerant..."
4,5,You provide smallholder farmers in India with ...,"If you cannot get certified maize seeds, you c..."
5,6,You provide smallholder farmers in India with ...,Chemical pesticides are important for controll...
6,7,You provide smallholder farmers in India with ...,"To check if your soil is fertile, observe thes..."
7,8,You provide smallholder farmers in India with ...,"To control weeds effectively, use a mix of met..."
8,9,You provide smallholder farmers in India with ...,"Organic fertilizers (like compost, manure) imp..."
9,10,You provide smallholder farmers in India with ...,"Given the uncertainty in monsoon onset, follow..."


In [ ]:
deepseek_r1_df = get_llm_answers(qa_df, "deepseek/deepseek-reasoner", country, max_completion_tokens=25000)

Saving results to results/RAW_goldenqa_iteration1_oe_india_REVISED-deepseek-deepseek-reasoner.csv


In [24]:
deepseek_r1_df

,id,prompt,answer
0,1,You provide smallholder farmers in India with ...,Your pesticides may not work due to resistance...
1,2,You provide smallholder farmers in India with ...,To claim compensation for crop/livestock losse...
2,3,You provide smallholder farmers in India with ...,Visit your nearest **Krishi Vigyan Kendra (KVK...
3,4,You provide smallholder farmers in India with ...,"For dryland areas with low rainfall, consider ..."
4,5,You provide smallholder farmers in India with ...,"If you cannot get certified seeds, save seeds ..."
5,6,You provide smallholder farmers in India with ...,Chemical pesticides are important tools to con...
6,7,You provide smallholder farmers in India with ...,To check if your soil is fertile: \n1. **Colo...
7,8,You provide smallholder farmers in India with ...,To control weeds effectively: \n1. **Remove w...
8,9,You provide smallholder farmers in India with ...,"Organic fertilizers (like compost, manure) rel..."
9,10,You provide smallholder farmers in India with ...,The best time to plant depends on soil moistur...


# Gemini

In [ ]:
gemini_2_5_pro_df = get_llm_answers(qa_df, "gemini/gemini-2.5-pro-preview-05-06", country, max_completion_tokens=25000, reasoning_effort="medium")

Saving results to results/RAW_goldenqa_iteration1_oe_india_REVISED-gemini-gemini-2.5-pro-preview-05-06.csv


In [26]:
gemini_2_5_pro_df

,id,prompt,answer
0,1,You provide smallholder farmers in India with ...,"Namaste. This is a common problem, and it ofte..."
1,2,You provide smallholder farmers in India with ...,Losing your crops and livestock to extreme wea...
2,3,You provide smallholder farmers in India with ...,"Namaste! That's a great question, it's always ..."
3,4,You provide smallholder farmers in India with ...,"Namaste! For dryland areas, several hardy bean..."
4,5,You provide smallholder farmers in India with ...,"Namaste! If you cannot find certified seeds, y..."
5,6,You provide smallholder farmers in India with ...,Namaste! Chemical pesticides can be very helpf...
6,7,You provide smallholder farmers in India with ...,Namaste! Healthy crops with good yields are of...
7,8,You provide smallholder farmers in India with ...,Namaste! Controlling weeds is very important f...
8,9,You provide smallholder farmers in India with ...,Namaste! Both organic and synthetic fertilizer...
9,10,You provide smallholder farmers in India with ...,Namaste. This is a common challenge many farme...


# Aggregate results

In [81]:
basename

'RAW_goldenqa_iteration1_oe_india_REVISED'

In [82]:
import glob

# Get a list of all CSV files in the results directory
csv_files = glob.glob(f"results/{basename}-*.csv")
# Print the list of CSV files
print(csv_files)

['results/RAW_goldenqa_iteration1_oe_india_REVISED-deepseek-deepseek-reasoner.csv', 'results/RAW_goldenqa_iteration1_oe_india_REVISED-openai-gpt-4o-2024-08-06.csv', 'results/RAW_goldenqa_iteration1_oe_india_REVISED-anthropic-claude-3-7-sonnet-20250219.csv', 'results/RAW_goldenqa_iteration1_oe_india_REVISED-openai-o1-2024-12-17.csv', 'results/RAW_goldenqa_iteration1_oe_india_REVISED-anthropic-claude-3-5-sonnet-20241022.csv', 'results/RAW_goldenqa_iteration1_oe_india_REVISED-openai-o4-mini-2025-04-16.csv', 'results/RAW_goldenqa_iteration1_oe_india_REVISED-gemini-gemini-2-5-pro-preview-05-06.csv', 'results/RAW_goldenqa_iteration1_oe_india_REVISED-deepseek-deepseek-chat.csv', 'results/RAW_goldenqa_iteration1_oe_india_REVISED-openai-o3-2025-04-16.csv']


In [83]:
# read all csv files, extract the model name from the filename, and concatenate them into a single dataframe
dfs = []
for file in csv_files:
    model_name = "-".join(os.path.basename(file).split("-")[1:]).split(".")[0]
    df = pd.read_csv(file)
    df["model"] = model_name
    dfs.append(df)

# concatenate all dataframes into a single dataframe
all_df = pd.concat(dfs, ignore_index=True)

# drop the prompt column
all_df = all_df.drop(columns=["prompt"])

# add country column
all_df["country"] = country

In [84]:
all_df

,id,answer,model,country
0,1,Your pesticides may not work due to resistance...,deepseek-deepseek-reasoner,India
1,2,To claim compensation for crop/livestock losse...,deepseek-deepseek-reasoner,India
2,3,Visit your nearest **Krishi Vigyan Kendra (KVK...,deepseek-deepseek-reasoner,India
3,4,"For dryland areas with low rainfall, consider ...",deepseek-deepseek-reasoner,India
4,5,"If you cannot get certified seeds, save seeds ...",deepseek-deepseek-reasoner,India
...,...,...,...,...
130,11,1. Nitrogen lack shows as pale-green to yellow...,openai-o3-2025-04-16,India
131,12,1. Harvest at the right maturity in the cool m...,openai-o3-2025-04-16,India
132,13,1. Select short-duration or drought/heat-toler...,openai-o3-2025-04-16,India
133,14,1. Buy animals only from trusted sources and k...,openai-o3-2025-04-16,India


In [85]:
# add the questions by merging with the original dataframe
all_df.rename(columns={"id": "question_id"}, inplace=True)
all_df = all_df.merge(qa_df[["question_id", "question"]], on="question_id", how="left")

In [86]:
all_df

,question_id,answer,model,country,question
0,1,Your pesticides may not work due to resistance...,deepseek-deepseek-reasoner,India,I have been applying different pesticides to c...
1,2,To claim compensation for crop/livestock losse...,deepseek-deepseek-reasoner,India,I have been losing my crops and livestock due ...
2,3,Visit your nearest **Krishi Vigyan Kendra (KVK...,deepseek-deepseek-reasoner,India,I am interested in learning about crops and li...
3,4,"For dryland areas with low rainfall, consider ...",deepseek-deepseek-reasoner,India,Which bean variety is suitable for dryland agr...
4,5,"If you cannot get certified seeds, save seeds ...",deepseek-deepseek-reasoner,India,If I am unable to acquire certified maize seed...
...,...,...,...,...,...
130,11,1. Nitrogen lack shows as pale-green to yellow...,openai-o3-2025-04-16,India,What are the signs of nutrient deficiency in m...
131,12,1. Harvest at the right maturity in the cool m...,openai-o3-2025-04-16,India,How can I mitigate post-harvest losses in crop...
132,13,1. Select short-duration or drought/heat-toler...,openai-o3-2025-04-16,India,What are some of the strategies that I can emp...
133,14,1. Buy animals only from trusted sources and k...,openai-o3-2025-04-16,India,What are the best disease management practices...


In [87]:
# add a response_id column
all_df["response_id"] = all_df.index + 1

In [88]:
all_df

,question_id,answer,model,country,question,response_id
0,1,Your pesticides may not work due to resistance...,deepseek-deepseek-reasoner,India,I have been applying different pesticides to c...,1
1,2,To claim compensation for crop/livestock losse...,deepseek-deepseek-reasoner,India,I have been losing my crops and livestock due ...,2
2,3,Visit your nearest **Krishi Vigyan Kendra (KVK...,deepseek-deepseek-reasoner,India,I am interested in learning about crops and li...,3
3,4,"For dryland areas with low rainfall, consider ...",deepseek-deepseek-reasoner,India,Which bean variety is suitable for dryland agr...,4
4,5,"If you cannot get certified seeds, save seeds ...",deepseek-deepseek-reasoner,India,If I am unable to acquire certified maize seed...,5
...,...,...,...,...,...,...
130,11,1. Nitrogen lack shows as pale-green to yellow...,openai-o3-2025-04-16,India,What are the signs of nutrient deficiency in m...,131
131,12,1. Harvest at the right maturity in the cool m...,openai-o3-2025-04-16,India,How can I mitigate post-harvest losses in crop...,132
132,13,1. Select short-duration or drought/heat-toler...,openai-o3-2025-04-16,India,What are some of the strategies that I can emp...,133
133,14,1. Buy animals only from trusted sources and k...,openai-o3-2025-04-16,India,What are the best disease management practices...,134


In [89]:
# shuffle the dataframe
all_df = all_df.sample(frac=1, random_state=1337).reset_index(drop=True)
all_df

,question_id,answer,model,country,question,response_id
0,3,To learn more about crops and livestock produc...,openai-gpt-4o-2024-08-06,India,I am interested in learning about crops and li...,18
1,15,"Farmer groups, FPOs, cooperatives, and CBOs st...",deepseek-deepseek-reasoner,India,"How are Farmer groups, Farmer Producer Organiz...",15
2,6,# Chemical Pesticides in Farming: Important bu...,anthropic-claude-3-7-sonnet-20250219,India,Are chemical pesticides important and safe in ...,36
3,14,"1. Inspect your animals every day for cough, f...",openai-o4-mini-2025-04-16,India,What are the best disease management practices...,89
4,11,# Signs of Nutrient Deficiency in Maize and So...,anthropic-claude-3-7-sonnet-20250219,India,What are the signs of nutrient deficiency in m...,41
...,...,...,...,...,...,...
130,10,"1. Rather than fixed calendar dates, sow when ...",openai-o4-mini-2025-04-16,India,"Lately, the onset of rains has shown significa...",85
131,8,1. Begin with clean seed and avoid bringing we...,openai-o4-mini-2025-04-16,India,How can I best control weeds in my farm?,83
132,10,"Given the uncertainty in monsoon onset, follow...",deepseek-deepseek-chat,India,"Lately, the onset of rains has shown significa...",115
133,1,"Namaste. This is a common problem, and it ofte...",gemini-gemini-2-5-pro-preview-05-06,India,I have been applying different pesticides to c...,91


In [90]:
# reorder the columns
all_df = all_df[["model", "question_id", "response_id", "question", "answer", "country"]]
all_df

,model,question_id,response_id,question,answer,country
0,openai-gpt-4o-2024-08-06,3,18,I am interested in learning about crops and li...,To learn more about crops and livestock produc...,India
1,deepseek-deepseek-reasoner,15,15,"How are Farmer groups, Farmer Producer Organiz...","Farmer groups, FPOs, cooperatives, and CBOs st...",India
2,anthropic-claude-3-7-sonnet-20250219,6,36,Are chemical pesticides important and safe in ...,# Chemical Pesticides in Farming: Important bu...,India
3,openai-o4-mini-2025-04-16,14,89,What are the best disease management practices...,"1. Inspect your animals every day for cough, f...",India
4,anthropic-claude-3-7-sonnet-20250219,11,41,What are the signs of nutrient deficiency in m...,# Signs of Nutrient Deficiency in Maize and So...,India
...,...,...,...,...,...,...
130,openai-o4-mini-2025-04-16,10,85,"Lately, the onset of rains has shown significa...","1. Rather than fixed calendar dates, sow when ...",India
131,openai-o4-mini-2025-04-16,8,83,How can I best control weeds in my farm?,1. Begin with clean seed and avoid bringing we...,India
132,deepseek-deepseek-chat,10,115,"Lately, the onset of rains has shown significa...","Given the uncertainty in monsoon onset, follow...",India
133,gemini-gemini-2-5-pro-preview-05-06,1,91,I have been applying different pesticides to c...,"Namaste. This is a common problem, and it ofte...",India


In [91]:
# save the dataframe to a CSV file
all_df.to_csv(f"{basename}-all.csv", index=False)